In [1]:
%pip install langchain faiss-cpu transformers torch accelerate sentence-transformers dashscope

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install dashscope langchain-community

Looking in indexes: https://mirrors.tencent.com/pypi/simple/

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install python-docx PyPDF2 unstructured pdf2image pillow

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 1.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 1.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 1.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
%pip install docx2txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.tencent.com/pypi/simple/

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install langchain langchain-community docx2txt PyPDF2 concurrent.futures

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
ERROR: Could not find a version that satisfies the requirement concurrent.futures (from versions: none)
ERROR: No matching distribution found for concurrent.futures

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# -*- coding: utf-8 -*-
import os
import re
import glob
import concurrent.futures  # ✅ 添加这一行
from concurrent.futures import ThreadPoolExecutor
from langchain.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.llms import Tongyi  # 新版接口
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document



# ================== 1. 自定义切分政策文档 ==================
def split_policy_documents(text):
    # 使用正则表达式匹配政策编号格式（如 "一、"、"(一)"、"(二)" 等）
    pattern = r"(一|二|三|四|五|六|七|八|九|十、|$[一二三四五六七八九十]$)"
    sections = re.split(pattern, text)

    # 过滤掉空字符串并重新组合编号和内容
    result = []
    for i in range(1, len(sections), 2):
        if i + 1 < len(sections):
            title = sections[i] + sections[i + 1]
            title = title.strip()
            if title:
                result.append(title)
    
    # 限制最多提取 32 条政策
    return result[:32]


# ================== 2. 并行处理单个文件 ==================
def process_file(file_path):
    print(f"正在处理文件: {file_path}")
    try:
        if file_path.endswith(".txt"):
            loader = TextLoader(file_path, encoding="utf-8")
        elif file_path.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif file_path.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        else:
            raise ValueError(f"不支持的文件类型: {file_path}")

        # 加载文档内容
        data = loader.load()
        full_text = "\n\n".join([d.page_content for d in data])

        # 使用智能切分器（按段落、句子等）
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len,
        )
        chunks = text_splitter.split_text(full_text)
        return chunks
    except Exception as e:
        print(f"处理文件 {file_path} 时出错: {e}")
        return []


# ================== 3. 并行读取问题文件 ==================
def load_question_files(root_dir="."):
    questions = []

    # 获取所有用户输入文件（排除 laws.txt）
    file_paths = [
        f for f in glob.glob(os.path.join(root_dir, "*")) 
        if os.path.isfile(f) and not os.path.basename(f) == "laws.txt"
    ]

    # 并行处理文件
    with ThreadPoolExecutor(max_workers=4) as executor:
        future_to_file = {
            executor.submit(process_file, f): f for f in file_paths
        }

        for future in concurrent.futures.as_completed(future_to_file):
            result = future.result()
            questions.extend(result)

    return questions


# ================== 4. 加载并切分 laws.txt ==================
def load_and_split_documents(file_path="laws.txt"):
    loader = TextLoader(file_path, encoding="utf-8")
    documents = loader.load()
    
    # 提取原始文本
    raw_text = documents[0].page_content if documents else ""
    
    # 按政策编号切分
    policy_sections = split_policy_documents(raw_text)
    
    # 构建 Document 对象列表
    docs = [Document(page_content=section) for section in policy_sections]
    return docs


# ================== 5. 初始化 Embedding 模型 ==================
def get_embeddings():
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)
    return embeddings


# ================== 6. 构建向量数据库 ==================
def build_vectorstore(docs, embeddings):
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore


# ================== 7. 初始化 Qwen LLM（Tongyi）==================
def get_llm(api_key):
    llm = Tongyi(model_name="qwen-max", dashscope_api_key=api_key)
    return llm


# ================== 8. 定义 Prompt 模板 ==================
def get_prompt_template():
    template = """用户问题：{question}

相关法律条文：{context}

请按照以下格式输出：
        1. 结论（合规/部分合规/不合规）
        2. 理由（列出条款编号和理由） 
"""
    prompt = PromptTemplate(template=template, input_variables=["question", "context"])
    return prompt


# ================== 9. 构建 QA 链 ==================
def build_qa_chain(vectorstore, llm, prompt):
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 2}),
        chain_type_kwargs={"prompt": prompt}
    )
    return qa_chain


# ================== 10. 每条问题单独处理函数 ==================
def process_query(qa_chain, query):
    result = qa_chain.run(query)
    return {"question": query, "result": result}


# ================== 主函数入口 ==================
def main():
    # 设置 DashScope API Key
    DASHSCOPE_API_KEY = "sk-7fb2aee47f5d4531855a7ac3412249fe"  # 替换为你的实际 API Key
    os.environ["DASHSCOPE_API_KEY"] = DASHSCOPE_API_KEY

    # 步骤 1：加载并切分 laws.txt（构建向量数据库）
    print("正在加载并切分 laws.txt 用于构建向量数据库...")
    docs = load_and_split_documents()

    # 步骤 2：初始化 Embedding 模型
    print("正在加载 Embedding 模型...")
    embeddings = get_embeddings()

    # 步骤 3：构建 FAISS 向量数据库
    print("正在构建向量数据库...")
    vectorstore = build_vectorstore(docs, embeddings)

    # 步骤 4：初始化 LLM
    print("正在加载 Qwen 模型...")
    llm = get_llm(DASHSCOPE_API_KEY)

    # 步骤 5：定义 Prompt 模板
    print("正在构建 Prompt 模板...")
    prompt = get_prompt_template()

    # 步骤 6：构建 QA 链
    print("正在整合 RetrievalQA 链...")
    qa_chain = build_qa_chain(vectorstore, llm, prompt)

    # 步骤 7：读取用户输入的问题文件（并行处理）
    print("\n正在读取用户输入的问题文件...")
    queries = load_question_files()

    print(f"\n共读取到 {len(queries)} 条问题，开始并行推理...\n")

    results = []
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = [executor.submit(process_query, qa_chain, q) for q in queries]
        for future in futures:
            results.append(future.result())

    # 输出结果
    print("\n=== 推理完成 ===\n")
    for item in results:
        print("【问题】", item["question"])
        print("【回答】", item["result"])
        print("-" * 50)


if __name__ == "__main__":
    main()

正在加载并切分 laws.txt 用于构建向量数据库...
正在加载 Embedding 模型...


/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


正在构建向量数据库...
正在加载 Qwen 模型...
正在构建 Prompt 模板...
正在整合 RetrievalQA 链...

正在读取用户输入的问题文件...
正在处理文件: ./demo2.ipynb
处理文件 ./demo2.ipynb 时出错: 不支持的文件类型: ./demo2.ipynb
正在处理文件: ./legal_index.faiss
处理文件 ./legal_index.faiss 时出错: 不支持的文件类型: ./legal_index.faiss
正在处理文件: ./README.md
处理文件 ./README.md 时出错: 不支持的文件类型: ./README.md
正在处理文件: ./政策四.txt
正在处理文件: ./demo1.ipynb
处理文件 ./demo1.ipynb 时出错: 不支持的文件类型: ./demo1.ipynb
正在处理文件: ./政策三.docx
正在处理文件: ./demo3.ipynb
处理文件 ./demo3.ipynb 时出错: 不支持的文件类型: ./demo3.ipynb
正在处理文件: ./jieshi.ipynb
处理文件 ./jieshi.ipynb 时出错: 不支持的文件类型: ./jieshi.ipynb
正在处理文件: ./政策一.docx
正在处理文件: ./demo final.ipynb
处理文件 ./demo final.ipynb 时出错: 不支持的文件类型: ./demo final.ipynb
正在处理文件: ./demo4.ipynb
处理文件 ./demo4.ipynb 时出错: 不支持的文件类型: ./demo4.ipynb
正在处理文件: ./政策二.txt

共读取到 25 条问题，开始并行推理...


=== 推理完成 ===

【问题】 瑞安市人民政府关于进一步深化企业减负担降成本的若干意见
发布日期：2018.06.01 制定机关：瑞安市人民政府
【回答】 ### 1. 结论：部分合规

### 2. 理由：

- **条款一）为维护国家安全和发展利益的**
  - **理由**：该条款的内容与瑞安市人民政府发布的《关于进一步深化企业减负担降成本的若干意见》并不直接冲突。文件的主要目的是通过减轻企业负担和降低运营成本来促进经济发展，这在一

In [1]:
import faiss
import numpy as np

# 1. 加载 .faiss 索引文件
index = faiss.read_index("legal_index.faiss")

# 2. 查看索引信息
print(f"维度: {index.d}")
print(f"当前向量数量: {index.ntotal}")

# 3. (可选) 查询一些最近邻
# 构造一个随机查询向量（维数要和索引一致）
query_vector = np.random.random((1, index.d)).astype('float32')

# 搜索最相似的 5 个向量
distances, indices = index.search(query_vector, 5)

print("距离:", distances)
print("索引位置:", indices)

维度: 384
当前向量数量: 6
距离: [[129.42017 130.18298 130.26993 131.06049 131.59166]]
索引位置: [[4 3 2 1 0]]


In [5]:
%pip install pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 10.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        # 提取文本
        page_text = page.get_text("text")
        text += page_text + "\n\n"
    return text

# 使用示例
policy_text = extract_text_from_pdf("政策二.pdf")
print(policy_text)

中共龙港市委、龙港市政府办公室关于进一步深化企业减负担降成本
改革的实施意见
发布日期：2021.11.11
制定机关：中共龙港市委员会
市直属各单位：
为进一步减轻企业各类负担和成本，深化一流营商环境建设，有效激发市场主体活力，促进经济
持续健康发展，根据《温州市人民政府办公室关于进一步深化企业减负担降成本改革的实施意见》（温
政办〔2021〕58 号）精神，结合我市实际，提出以下实施意见：
一、实施小微企业普惠性优惠政策。（一）增值税小规模纳税人月销售额15 万元（含本数）以
下免税。小规模纳税人增值税起征点从月销售额10 万元提高到15 万元（含本数）。执行时间为2021
年4 月1 日至2022 年12 月31 日。（二）延续实施小规模纳税人减半征收“六税两费”。对增值
税小规模纳税人，按照50%幅度减征资源税、城市维护建设税、房产税、城镇土地使用税、印花税、
耕地占用税和教育费附加、地方教育附加。执行时间至2021 年12 月31 日。（三）对小型微利企业
减免企业所得税。对小型微利企业年应纳税所得额不超过100 万元的部分，在现行优惠政策基础上，
再减半征收所得税。执行时间为2021 年1 月1 日至2022 年12 月31 日。（责任单位：市税务局）
二、延续实施普惠金融税收政策。对金融机构向农户、小微企业及个体工商户发放小额贷款取得
的利息收入，免征增值税；对小额贷款公司取得的农户小额贷款利息收入，免征增值税；对金融机构、
小额贷款公司取得的农户小额贷款的利息收入，在计算企业所得税应纳税所得额时，按90%计入收
入总额。执行时间至2023 年12 月31 日。（责任单位：市税务局）
三、增值税增量留抵退税。对先进制造业企业按月全额退还增值税增量留抵税额。自2021 年4
月1 日起，将运输设备、电气机械和器材、仪器仪表、医药、化学纤维等制造业企业纳入先进制造业
企业增值税增量留抵退税政策范围。执行时间为长期。（责任单位：市税务局）
四、延续并优化研发费用加计扣除政策。延续执行企业研发费用加计扣除75%政策，执行时间
至2023 年12 月31 日；将制造业企业加计扣除比例进一步提高到100%；改革研发费用加计扣除清


缴核算方式，允许企业自主选择按前三季度享受加计扣除优惠，前三季度的研发费用由次年所得税汇
算清缴时扣除改为当年第3 季度（按季预缴）

In [8]:
%pip install pdfplumber

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 526.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 9.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 3.9 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import pdfplumber
import re

def is_sentence_complete(text):
    """
    判断文本是否以完整句子结尾（以句号、问号、感叹号结尾）
    """
    if not text:
        return False
    # 匹配中文或英文的句子结束符
    pattern = r"[。！？\.\!\?]$"
    return bool(re.search(pattern, text.strip()))

def merge_incomplete_pages(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        total_pages = len(pdf.pages)
        blocks = []  # 存储合并后的内容块
        i = 0

        while i < total_pages:
            page = pdf.pages[i]
            text = page.extract_text()
            if not text:
                i += 1
                continue

            # 如果当前页不完整，合并下一页
            if i + 1 < total_pages and not is_sentence_complete(text):
                next_page = pdf.pages[i + 1]
                next_text = next_page.extract_text()
                merged_text = text + next_text
                # 递归检查合并后的内容是否完整
                if not is_sentence_complete(merged_text):
                    i += 1
                    merged_text += next_page.extract_text()
                i += 1  # 跳过已处理的下一页
            else:
                merged_text = text

            # 分割为块（按段落或句子）
            paragraphs = re.split(r"\n{2,}", merged_text.strip())  # 按两个换行分割段落
            for p in paragraphs:
                if p.strip():  # 去除空块
                    blocks.append(p.strip())

            i += 1

        return blocks

# 示例：处理 PDF 并显示合并后的内容块
pdf_path = "政策二.pdf"
blocks = merge_incomplete_pages(pdf_path)

# 显示前5个块
for idx, block in enumerate(blocks[:5], 1):
    print(f"【块 {idx}】\n{block}\n{'-' * 50}")

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


【块 1】
中共龙港市委、龙港市政府办公室关于进一步深化企业减负担降成本
改革的实施意见
发布日期：2021.11.11 制定机关：中共龙港市委员会
市直属各单位：
为进一步减轻企业各类负担和成本，深化一流营商环境建设，有效激发市场主体活力，促进经济
持续健康发展，根据《温州市人民政府办公室关于进一步深化企业减负担降成本改革的实施意见》（温
政办〔2021〕58号）精神，结合我市实际，提出以下实施意见：
一、实施小微企业普惠性优惠政策。（一）增值税小规模纳税人月销售额15万元（含本数）以
下免税。小规模纳税人增值税起征点从月销售额10万元提高到15万元（含本数）。执行时间为2021
年4月1日至2022年12月31日。（二）延续实施小规模纳税人减半征收“六税两费”。对增值
税小规模纳税人，按照50%幅度减征资源税、城市维护建设税、房产税、城镇土地使用税、印花税、
耕地占用税和教育费附加、地方教育附加。执行时间至2021年12月31日。（三）对小型微利企业
减免企业所得税。对小型微利企业年应纳税所得额不超过100万元的部分，在现行优惠政策基础上，
再减半征收所得税。执行时间为2021年1月1日至2022年12月31日。（责任单位：市税务局）
二、延续实施普惠金融税收政策。对金融机构向农户、小微企业及个体工商户发放小额贷款取得
的利息收入，免征增值税；对小额贷款公司取得的农户小额贷款利息收入，免征增值税；对金融机构、
小额贷款公司取得的农户小额贷款的利息收入，在计算企业所得税应纳税所得额时，按90%计入收
入总额。执行时间至2023年12月31日。（责任单位：市税务局）
三、增值税增量留抵退税。对先进制造业企业按月全额退还增值税增量留抵税额。自2021年4
月1日起，将运输设备、电气机械和器材、仪器仪表、医药、化学纤维等制造业企业纳入先进制造业
企业增值税增量留抵退税政策范围。执行时间为长期。（责任单位：市税务局）
四、延续并优化研发费用加计扣除政策。延续执行企业研发费用加计扣除75%政策，执行时间
至2023年12月31日；将制造业企业加计扣除比例进一步提高到100%；改革研发费用加计扣除清缴核算方式，允许企业自主选择按前三季度享受加计扣除优惠，前三季度的研发费用由次年所得税汇
算清缴时扣除改为当年第3季度（按季预缴）或9月份（按月预缴）时即可扣除。（责任单位：市税
